In [ ]:
## All the external libraries
import re
import csv
from getpass import getpass
from time import sleep
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
#from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver import Chrome

In [ ]:
## All the internal libraries

In [ ]:
####### All the functions #######

## Get random time offset
def get_rand_offset(t=4):
    r = random.randint(0, t)
    r += random.random()
    return r

#### Get the browser
## Return <class 'selenium.webdriver.chrome.webdriver.WebDriver'>
## Opens an empty browser
def get_driver():
    if driver == None:
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    else:
        return driver

## Go to address
def visit_site(site="https://www.google.com"):
    driver.get(site)
    
## Add tweets to file
def write_to_file(fname, data):
    pass

## Get n tweets
def get_ntweets(num_tweets=100):
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        page_cards = []
        while len(page_cards) < 1:
            page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')

        min_tweets = -1*min(len(page_cards), 15)

        for card in page_cards[min_tweets:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    if len(data) == num_tweets:
                        scrolling = False
                        break
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1+get_rand_offset(2))
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(1+get_rand_offset(2)) # attempt another scroll
            else:
                last_position = curr_position
                break
    return data

## Get specific date tweets
def get_specific_date_tweets(search_term, date, num_tweets=100):
    url_to_visit = 'https://twitter.com/search?f=live&q=(%23Covid19)%20lang%3Aen
    '%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query'
    visit_site(site=url_to_visit)
    tweets = get_ntweets(num_tweets=num_tweets)
    write_to_file(fname=fname, data=tweets)
    pass

    

## Description
## Takes
## Returns
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    # get a string of all emojis contained in the tweet
    """Emojis are stored as images... so I convert the filename, which is stored as unicode, into 
    the emoji character."""
    emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt)
    return tweet   

In [ ]:
## Initialization: To be run only once at the begining
driver = None

In [ ]:
## Get the browser
driver = get_driver()

In [ ]:
## Webdriver support following
print(f'dir of webdriver: {dir(driver)}')

In [ ]:
## Visit Google
visit_site()

In [ ]:
## Visit twitter
visit_site(site='https://twitter.com/')

In [ ]:
twitter_cookies = driver.get_cookies()

In [ ]:
print(twitter_cookies)

In [ ]:
## Deletes the current cookies
driver.delete_all_cookies()

In [ ]:
## Restore the cookie
for cookie in twitter_cookies:
    #print(f'cookie:{cookie}')
    driver.add_cookie(cookie_dict=cookie)

In [ ]:
## Refreshes the page
driver.refresh()

In [ ]:
## Search
visit_site(site='https://twitter.com/search?f=live&q=(%23Covid19)%20lang%3Aen%20until%3A2020-05-02%20since%3A2020-05-01&src=typed_query')

In [ ]:
# get all tweets on the page
data = get_ntweets(num_tweets=5)

In [ ]:
print(data)

In [ ]:
len(data)